In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import sys

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

In [3]:
P = 32771

In [4]:
np.random.seed(P)
random_sample = list(set(np.random.randint(0, 2**32, size=4000000).tolist()))
np.random.shuffle(random_sample)
len(random_sample), np.array(random_sample).sum()

(3998155, 8583390742850828)

In [5]:
def dataset(xs):
    X = pd.DataFrame(map(lambda x: map(int,f'{x:032b}'), xs))
    y = pd.Series(map(lambda x: (x % P) % 2, xs))
    return X, y

In [6]:
X_train, y_train = dataset(random_sample[:2000000])
X_val, y_val = dataset(random_sample[-400000:-200000])
X_test, y_test = dataset(random_sample[-200000:])

In [7]:
print ('train', X_train.shape, y_train.shape)
print ('val', X_val.shape, y_val.shape)
print ('test', X_test.shape, y_test.shape)

train (2000000, 32) (2000000,)
val (200000, 32) (200000,)
test (200000, 32) (200000,)


In [8]:
inputs = Input(shape=(32,), name='digits')
x = Dense(64, activation='relu', name='dense_1')(inputs)
x = Dense(64, activation='relu', name='dense_2')(x)
outputs = Dense(1, activation='sigmoid', name='predictions')(x)

model = Model(inputs=inputs, outputs=outputs)

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_val, y_val))

Train on 2000000 samples, validate on 200000 samples
Epoch 1/3
2000000/2000000 [==============================] - 35s 18us/sample - loss: 0.1070 - accuracy: 0.9459 - val_loss: 0.0317 - val_accuracy: 0.9868
Epoch 2/3
2000000/2000000 [==============================] - 33s 17us/sample - loss: 0.0270 - accuracy: 0.9887 - val_loss: 0.0358 - val_accuracy: 0.9851
Epoch 3/3
2000000/2000000 [==============================] - 33s 17us/sample - loss: 0.0225 - accuracy: 0.9906 - val_loss: 0.0203 - val_accuracy: 0.9912


In [12]:
th = 0.5

print ('train', ((model.predict(X_train)[:,0] > th) == y_train.values).mean())
print ('val', ((model.predict(X_val)[:,0] > th) == y_val.values).mean())
print ('test', ((model.predict(X_test)[:,0] > th) == y_test.values).mean())

train 0.9915235
val 0.991185
test 0.991135
